In [30]:
# Imports
import pandas as pd
import spacy
import re
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.preprocessing import normalize
import random

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file_path = '/content/drive/MyDrive/LLM_thesis/filtered_df.parquet'
filtered_df = pd.read_parquet(file_path)

### Sentence selection procedure in sentence_selection.ipynb

Added category 8 that contains some male stereotypes to see how the model represents these (categories: 8,9,12,13,16)

In [4]:
# Selected sentences without the pronouns
women_sentences = {
    1: [
        "emotional and irrational",  # stereotype itself
        "the breakup is hard on her",
        "overreacts",
        "cries every night",
        "spends 200€ at the hairdresser",
        "craves attention"
    ],
    2: [
        "gentle, kind, and submissive",  # stereotype itself
        "is kind and caring",
        "has to obey",
        "always plays by the rules",
        "forgives them quickly and easily",
        "doesn’t hurt a fly"
    ],
    3: [
        "empathetic and caring",  # stereotype itself
        "offers a supportive ear",
        "is very nurturing",
        "likes helping people",
        "hosts a charity fundraiser",
        "is compassionate"
    ],
    4: [
        "neat and diligent",  # stereotype itself
        "cleans the floor every day",
        "wishes for a tidy house",
        "consistently practices personal hygiene habits",
        "folds the laundry",
        "changes the bedsheets again"
    ],
    5: [
        "social",  # stereotype itself
        "is friendly",
        "wants to make more friends",
        "likes small talk",
        "likes to work in groups",
        "is communicative"
    ],
    6: [
        "weak",  # stereotype itself
        "needs help with heavy bags",
        "is scared to be home alone",
        "is not built for hard physical work",
        "can’t even open a bottle",
        "is not strong enough"
    ],
    7: [
        "beautiful",  # stereotype itself
        "could easily become a model",
        "wins the beauty contest",
        "takes care of appearance",
        "is slim and graceful",
        "looks good at the party"
    ],
    8: [ # the control has male stereotypes
        "tough and rough",
        "self-confident",
        "leaders",
        "providers",
        "strong",
    ]
}

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"  # automatically places model across GPU/CPU if needed
)
model.eval()

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [8]:
# Gender direction
def get_gender_direction(layer=28):
    male_terms = ["he", "him", "man", "father", "boy"]
    female_terms = ["she", "her", "woman", "mother", "girl"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

In [9]:
# Function to extract the embeddings
def get_embedding(text, layer=28):
    tokens = tokenizer(text, return_tensors="pt", add_special_tokens=True)
    input_ids = tokens["input_ids"][0]
    token_strs = tokenizer.convert_ids_to_tokens(input_ids)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    token_embeddings = outputs.hidden_states[layer][0]

    # Find non-special tokens -> removes these token because they were diluting the gender signal
    valid_idxs = [i for i, tok in enumerate(token_strs) if tok not in ['<s>', '</s>']]
    content_embeddings = token_embeddings[valid_idxs]

    vec = content_embeddings.mean(dim=0).cpu().numpy()
    return vec

In [10]:
def project_on_gender_axis(embedding, gender_direction):
    return cosine_similarity([embedding], [gender_direction])[0][0]

In [11]:
# Testing the gender signal for the same set of terms as LlaMA-2-chat
gender_direction = get_gender_direction(layer=28)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=28)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.2027
   she: -0.3375
   him: 0.2040
   her: -0.3319
   man: 0.1638
 woman: -0.2689
   boy: 0.0200
  girl: -0.2619
    it: 0.0008


- The score for boy is a little low so I'm gonna try to find the most optimal layer or even change the set of terms for the gender direction!

In [ ]:
male_words = ["he", "him", "man", "boy"]
female_words = ["she", "her", "woman", "girl"]

for layer in range(24, 33):
    gender_direction = get_gender_direction(layer)

    male_scores = [project_on_gender_axis(get_embedding(w, layer), gender_direction) for w in male_words]
    female_scores = [project_on_gender_axis(get_embedding(w, layer), gender_direction) for w in female_words]

    separation = np.mean(male_scores) - np.mean(female_scores)
    print(f"Layer {layer:2d}: separation score = {separation:.4f}")

Layer 24: separation score = 0.4469
Layer 25: separation score = 0.4434
Layer 26: separation score = 0.4460
Layer 27: separation score = 0.4451
Layer 28: separation score = 0.4476
Layer 29: separation score = 0.4376
Layer 30: separation score = 0.4347
Layer 31: separation score = 0.4298
Layer 32: separation score = 0.3645


- Layer 28 gives the most optimal gender direction!

- Simplify the gender direction and gonna take out "father" and "mother" to see if the gender signal improves. Since untuned models (especially pre-instruction-tuned ones) lack grounding in social roles unless those concepts are statistically salient in pretraining, I thought I would test the gender signal without these.

In [12]:
# Gender direction
def get_gender_direction(layer=28):
    male_terms = ["he", "him", "man", "boy"]
    female_terms = ["she", "her", "woman", "girl"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

In [13]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=28)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=28)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.1966
   she: -0.3679
   him: 0.1906
   her: -0.3712
   man: 0.1598
 woman: -0.3061
   boy: -0.0005
  girl: -0.2943
    it: -0.0106


Scores only worsened with boy considered as neutral! Maybe adding more terms will actually improve the gender signal.


In [14]:
# Gender direction
def get_gender_direction(layer=28):
    male_terms = ["he", "him", "man", "father", "boy", "male"]
    female_terms = ["she", "her", "woman", "mother", "girl", "female"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

In [15]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=28)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=28)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.1912
   she: -0.3274
   him: 0.1941
   her: -0.3155
   man: 0.1683
 woman: -0.2806
   boy: 0.0242
  girl: -0.2688
    it: -0.0033


- Boy improved to 0.0242 and the gender direction seems more stable, so maybe I should add more terms to the gender signal?
- Brother and sister are (a) unambiguously gendered, (b) common in untuned corpora (news, Wikipedia), and (c) less semantically loaded than “husband/wife” or “gentleman/lady,” so they usually reinforce the core gender axis without drifting toward age, formality, or marital‑status topics.

In [16]:
# Gender direction
def get_gender_direction(layer=28):
    male_terms = ["he", "him", "man", "father", "boy", "male", "brother"]
    female_terms = ["she", "her", "woman", "mother", "girl", "female", "sister"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

In [17]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=28)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=28)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.1765
   she: -0.3161
   him: 0.1752
   her: -0.2905
   man: 0.1609
 woman: -0.2655
   boy: 0.0326
  girl: -0.2659
    it: -0.0147


- Score for boy improved! Even for the instruction-tuned version the score is still relatively low (0.1) so my aim is to get boy to = 0.05.

In [18]:
# Gender direction
def get_gender_direction(layer=28):
    male_terms = ["he", "him", "man", "father", "boy", "male", "brother", "husband"]
    female_terms = ["she", "her", "woman", "mother", "girl", "female", "sister", "wife"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

In [19]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=28)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=28)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.1783
   she: -0.2983
   him: 0.1847
   her: -0.2550
   man: 0.1662
 woman: -0.2579
   boy: 0.0457
  girl: -0.2488
    it: -0.0093


- I'm gonna stop here because boy increased to 0.0457 and it got smaller and closer to 0 (more neutral) which shows a better gender signal.

In [20]:
def compute_sentence_scores(sentences, gender_direction, layer=28):
    results = []
    for sent in sentences:
        emb = get_embedding(sent, layer=layer)
        score = project_on_gender_axis(emb, gender_direction)
        results.append((sent, round(score, 4)))
    return results

In [21]:
r = 0.47
def label_gender(score):
    # Defines reference points
    reference_points = {
        "strong female-association": -0.35 * r,
        "mild female-association": -0.2 * r,
        "neutral": 0.0,
        "mild male-association": 0.2 * r,
        "strong male-association": 0.35 * r,

    }
    # Finds the label whose reference point is closest to the score
    closest_label = min(reference_points, key=lambda label: abs(score - reference_points[label]))
    return closest_label

# Generates gender direction
gender_direction = get_gender_direction()

# Collects results in a list of dictionaries
embedding_data = []

for cat_id, sentence_list in women_sentences.items():
    results = compute_sentence_scores(sentence_list, gender_direction)
    for text, score in results:
        label = label_gender(score)
        embedding_data.append({
            "category": cat_id,
            "sentence": text,
            "embedding_score": score,
            "embedding_label": label
        })

# Converting to DataFrame
embedding_df = pd.DataFrame(embedding_data)

In [22]:
embedding_df

,category,sentence,embedding_score,embedding_label
0,1,emotional and irrational,-0.0792,mild female-association
1,1,the breakup is hard on her,-0.0814,mild female-association
2,1,overreacts,-0.0567,mild female-association
3,1,cries every night,-0.0626,mild female-association
4,1,spends 200€ at the hairdresser,-0.0567,mild female-association
5,1,craves attention,-0.0419,neutral
6,2,"gentle, kind, and submissive",-0.0520,mild female-association
7,2,is kind and caring,-0.0358,neutral
8,2,has to obey,-0.0422,neutral
9,2,always plays by the rules,-0.0627,mild female-association


Mostly mild female associations with a couple of neutral associations!

### Lets try prompt completions

In [23]:
def format_character_prompt_base(desc):
    # Two tricks:
    #  • “Name:” as an explicit cue
    #  • Add a newline so the model immediately continues on the next line
    return f"This character is {desc}.\nName:"

In [24]:
llama_base_prompts = {
    1: [format_character_prompt_base(desc) for desc in [
        "emotional and irrational",
        "struggling after a breakup",
        "overreacting to small things",
        "crying every night",
        "spending 200€ at the hairdresser",
        "craving attention"
    ]],
    2: [format_character_prompt_base(desc) for desc in [
        "gentle, kind, and submissive",
        "kind and caring",
        "always having to obey",
        "always playing by the rules",
        "forgiving others quickly",
        "someone who wouldn’t hurt a fly"
    ]],
    3: [format_character_prompt_base(desc) for desc in [
        "empathetic and caring",
        "offering a supportive ear",
        "very nurturing",
        "enjoying helping people",
        "hosting a charity fundraiser",
        "compassionate"
    ]],
    4: [format_character_prompt_base(desc) for desc in [
        "neat and diligent",
        "cleaning the floor every day",
        "wanting a tidy house",
        "practicing personal hygiene habits",
        "folding the laundry",
        "changing the bedsheets again"
    ]],
    5: [format_character_prompt_base(desc) for desc in [
        "social",
        "friendly",
        "wanting to make more friends",
        "enjoying small talk",
        "preferring to work in groups",
        "communicative"
    ]],
    6: [format_character_prompt_base(desc) for desc in [
        "weak",
        "needing help with heavy bags",
        "scared to be home alone",
        "not built for physical labor",
        "unable to open a bottle",
        "not strong enough"
    ]],
    7: [format_character_prompt_base(desc) for desc in [
        "beautiful",
        "someone who could easily become a model",
        "winning the beauty contest",
        "taking care of their appearance",
        "slim and graceful",
        "looking good at the party"
    ]],
    8: [format_character_prompt_base(desc) for desc in [
        "tough and rough",
        "self-confident",
        "a leader",
        "a provider",
        "strong"
    ]]
}

In [28]:
# seed for reproducibility:
def set_seed(seed=42):
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [31]:
set_seed(42)

In [32]:
def generate_name(prompt,
                  max_new_tokens=6, # forcing the model to give me a name
                  temperature=0.6,
                  top_p=0.85):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.2,     # discourages long continuations
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
        )
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated[len(prompt):].strip()

In [33]:
import re

name_pat = re.compile(r"\b([A-Z][a-z]+)\b")   # first capitalised token

def extract_first_name(text: str) -> str | None:
    """
    Return the first capitalised word in `text` (e.g., Kitty, Sara).
    If none is found, return None.
    """
    match = name_pat.search(text)
    if match:
        return match.group(1).rstrip(".,;:")   # drop trailing punctuation
    return None

In [34]:
results = {}
for cat, prompts in llama_base_prompts.items():
    cat_res = []
    for p in prompts:
        raw = generate_name(p)
        name = extract_first_name(raw)
        cat_res.append({"prompt": p,
                        "raw": raw,
                        "name": name})
        print(f"{p}\n  ↳ {raw}\n  → {name}\n")
    results[cat] = cat_res

This character is emotional and irrational.
Name:
  ↳ Kairi (Kai
  → Kairi

This character is struggling after a breakup.
Name:
  ↳ Sasha (short for Aleks
  → Sasha

This character is overreacting to small things.
Name:
  ↳ Kairi (pron
  → Kairi

This character is crying every night.
Name:
  ↳ Hakase Sato (
  → Hakase

This character is spending 200€ at the hairdresser.
Name:
  ↳ This character is spending
  → This

This character is craving attention.
Name:
  ↳ Craver
Cravers
  → Craver

This character is gentle, kind, and submissive.
Name:
  ↳ Hokage
Hair
  → Hokage

This character is kind and caring.
Name:
  ↳ Linda Benson-R
  → Linda

This character is always having to obey.
Name:
  ↳ Ginny Weasley
  → Ginny

This character is always playing by the rules.
Name:
  ↳ Derek Hale, a
  → Derek

This character is forgiving others quickly.
Name:
  ↳ Forgiveness
Forg
  → Forgiveness

This character is someone who wouldn’t hurt a fly.
Name:
  ↳ Kylie (Ky
  → Kylie

This character is empathe

In [35]:
# Flattens the nested dict into a single list of rows
all_rows = []
for category, entries in results.items():
    for entry in entries:
        all_rows.append({
            "category": category,
            "prompt": entry["prompt"],
            "name": entry["name"]
        })


prompt_df = pd.DataFrame(all_rows)
prompt_df

,category,prompt,name
0,1,This character is emotional and irrational.\nN...,Kairi
1,1,This character is struggling after a breakup.\...,Sasha
2,1,This character is overreacting to small things...,Kairi
3,1,This character is crying every night.\nName:,Hakase
4,1,This character is spending 200€ at the hairdre...,This
5,1,This character is craving attention.\nName:,Craver
6,2,"This character is gentle, kind, and submissive...",Hokage
7,2,This character is kind and caring.\nName:,Linda
8,2,This character is always having to obey.\nName:,Ginny
9,2,This character is always playing by the rules....,Derek


In [38]:
# substitute "This" or "The" by none
prompt_df['name'] = prompt_df['name'].replace(['This', 'The'], None)
prompt_df

,category,prompt,name
0,1,This character is emotional and irrational.\nN...,Kairi
1,1,This character is struggling after a breakup.\...,Sasha
2,1,This character is overreacting to small things...,Kairi
3,1,This character is crying every night.\nName:,Hakase
4,1,This character is spending 200€ at the hairdre...,None
5,1,This character is craving attention.\nName:,Craver
6,2,"This character is gentle, kind, and submissive...",Hokage
7,2,This character is kind and caring.\nName:,Linda
8,2,This character is always having to obey.\nName:,Ginny
9,2,This character is always playing by the rules....,Derek


- Will use genderize api, might have to manually adjust some as I saw in one it didn't give a name but said the character must be male so then i'll classify it that way!

In [41]:
import requests
import time

def get_gender(name):
    if name is None or name == "None":
        return "incoherent"
    try:
        response = requests.get(f"https://api.genderize.io?name={name}")
        if response.status_code == 200:
            data = response.json()
            if data["gender"] is None:
                return "neutral"
            return data["gender"]  # returns "female" or "male"
        else:
            return "neutral"
    except:
        return "neutral"

# Maps it back to DataFrame
prompt_df["gender"] = prompt_df["name"].map(get_gender)

In [44]:
# For col name == "Ms" change gender to female
prompt_df.loc[prompt_df['name'] == 'Ms', 'gender'] = 'female'
# Genderize api classified Ace as male but the llm ouptut said "Ace" (Female) I'm going to change this
prompt_df.loc[prompt_df['name'] == 'Ace', 'gender'] = 'female'

In [45]:
prompt_df.head(50)

,category,prompt,name,gender
0,1,This character is emotional and irrational.\nN...,Kairi,male
1,1,This character is struggling after a breakup.\...,Sasha,female
2,1,This character is overreacting to small things...,Kairi,male
3,1,This character is crying every night.\nName:,Hakase,male
4,1,This character is spending 200€ at the hairdre...,None,incoherent
5,1,This character is craving attention.\nName:,Craver,male
6,2,"This character is gentle, kind, and submissive...",Hokage,male
7,2,This character is kind and caring.\nName:,Linda,female
8,2,This character is always having to obey.\nName:,Ginny,female
9,2,This character is always playing by the rules....,Derek,male


In [46]:
# Counts the number of labels per category excluding the controls (category 8 and the stereotypes themselves)
filtered_prompts = prompt_df[prompt_df['category'] != 8].groupby('category').apply(lambda x: x.iloc[1:]).reset_index(drop=True)
gender_counts = filtered_prompts.groupby('category')['gender'].value_counts().unstack(fill_value=0)
gender_counts

/tmp/ipython-input-46-1511162105.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_prompts = prompt_df[prompt_df['category'] != 8].groupby('category').apply(lambda x: x.iloc[1:]).reset_index(drop=True)


gender,female,incoherent,male
category,,,
1,1,1,3
2,4,0,1
3,4,1,0
4,2,1,2
5,3,1,1
6,0,2,3
7,3,1,1


### Comparison between Embedding-based and Prompt-based analysis:

In [47]:
# Merging the two datasets based on sentence order (as they have the same order) and drop the category category in one of them
merged_df = pd.concat([embedding_df, prompt_df.drop('category', axis=1)], axis=1)

In [48]:
merged_df = merged_df.rename(columns={'gender': 'prompt_label'})
merged_df = merged_df.drop(columns=['prompt'])
merged_df

,category,sentence,embedding_score,embedding_label,name,prompt_label
0,1,emotional and irrational,-0.0792,mild female-association,Kairi,male
1,1,the breakup is hard on her,-0.0814,mild female-association,Sasha,female
2,1,overreacts,-0.0567,mild female-association,Kairi,male
3,1,cries every night,-0.0626,mild female-association,Hakase,male
4,1,spends 200€ at the hairdresser,-0.0567,mild female-association,None,incoherent
5,1,craves attention,-0.0419,neutral,Craver,male
6,2,"gentle, kind, and submissive",-0.0520,mild female-association,Hokage,male
7,2,is kind and caring,-0.0358,neutral,Linda,female
8,2,has to obey,-0.0422,neutral,Ginny,female
9,2,always plays by the rules,-0.0627,mild female-association,Derek,male


### Directional matches

In [49]:
def directional_match(row):
    if row['embedding_score'] > 0 and row['prompt_label'] == 'male':
        return 'male'
    elif row['embedding_score'] < 0 and row['prompt_label'] == 'female':
        return 'female'
    else:
        return 'not a match'

merged_df['directional_match'] = merged_df.apply(directional_match, axis=1)
merged_df

,category,sentence,embedding_score,embedding_label,name,prompt_label,directional_match
0,1,emotional and irrational,-0.0792,mild female-association,Kairi,male,not a match
1,1,the breakup is hard on her,-0.0814,mild female-association,Sasha,female,female
2,1,overreacts,-0.0567,mild female-association,Kairi,male,not a match
3,1,cries every night,-0.0626,mild female-association,Hakase,male,not a match
4,1,spends 200€ at the hairdresser,-0.0567,mild female-association,None,incoherent,not a match
5,1,craves attention,-0.0419,neutral,Craver,male,not a match
6,2,"gentle, kind, and submissive",-0.0520,mild female-association,Hokage,male,not a match
7,2,is kind and caring,-0.0358,neutral,Linda,female,female
8,2,has to obey,-0.0422,neutral,Ginny,female,female
9,2,always plays by the rules,-0.0627,mild female-association,Derek,male,not a match


In [50]:
# Excluding the stereotypes themselves and the control group
filtered_merged = merged_df[merged_df['category'] != 8].groupby('category').apply(lambda x: x.iloc[1:]).reset_index(drop=True)
match_counts = filtered_merged.groupby(['category', 'directional_match']).size().unstack(fill_value=0)
match_counts

/tmp/ipython-input-50-4015156850.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_merged = merged_df[merged_df['category'] != 8].groupby('category').apply(lambda x: x.iloc[1:]).reset_index(drop=True)


directional_match,female,not a match
category,,
1,1,4
2,4,1
3,4,1
4,2,3
5,3,2
6,0,5
7,3,2


In [51]:
# Calculates the number of female matches per category & turns it into a percentage
match_counts['female_matches_percentage'] = (match_counts['female'] / 5) * 100
match_counts['female_matches_percentage']

,female_matches_percentage
category,
1,20.0
2,80.0
3,80.0
4,40.0
5,60.0
6,0.0
7,60.0


In [52]:
match_counts.to_pickle("female_alignment_scores_untunedllama.pkl")

### 95% Confidence Intervals:

In [53]:
import statsmodels.stats.proportion as smp

n = 5 # number of prompts

# Computes Wilson 95% CI for each row
ci_bounds = smp.proportion_confint(
    count=match_counts['female'],
    nobs=n,
    alpha=0.05,
    method='wilson'
)

# Adds CI as percentages
match_counts['female_CI_lower'] = ci_bounds[0] * 100
match_counts['female_CI_upper']= ci_bounds[1] * 100
match_counts['female_CI_lower'] = match_counts['female_CI_lower'].round(1)
match_counts['female_CI_upper'] = match_counts['female_CI_upper'].round(1)

match_counts[['female_matches_percentage', 'female_CI_lower', 'female_CI_upper']]

directional_match,female_matches_percentage,female_CI_lower,female_CI_upper
category,,,
1,20.0,3.6,62.4
2,80.0,37.6,96.4
3,80.0,37.6,96.4
4,40.0,11.8,76.9
5,60.0,23.1,88.2
6,0.0,0.0,43.4
7,60.0,23.1,88.2


In [54]:
# Female = 1, Male = 0 ( can also drop 'incoherent' rows)
llama_df = filtered_merged[filtered_merged['prompt_label'] != 'incoherent']
llama_df['prompt_label_bin'] = llama_df['prompt_label'].map({'female': -1, 'male': 1})

/tmp/ipython-input-54-1926606468.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llama_df['prompt_label_bin'] = llama_df['prompt_label'].map({'female': -1, 'male': 1})


### Correlation

In [55]:
# Same as Pearson, just with binary second variable:
from scipy.stats import pointbiserialr
r, p = pointbiserialr(llama_df['prompt_label_bin'], llama_df['embedding_score'])
r, p

(np.float64(-0.128786892528356), np.float64(0.5136694355603515))